In [259]:
%reload_ext autoreload
%autoreload 2
from EnzymePynetics.core import EnzymeKinetics
from EnzymePynetics.tools import ParameterEstimator

In [260]:
import pyenzyme as pe
from pathlib import Path

In [261]:
path = "/Users/max/code/EnzymePynetics/a-glucosidase.omex"
estimator = ParameterEstimator.from_EnzymeML(enzmldoc=path, measured_species_id="s1")

In [262]:
estimator.fit_models()

Fitting data to:
 - irreversible Michaelis Menten model
 -- Fitting succeeded: True
 - irreversible Michaelis Menten with enzyme inactivation model
 -- Fitting succeeded: True
 - competitive product inhibition model
 -- Fitting succeeded: True
 - competitive product inhibition with enzyme inactivation model
 -- Fitting succeeded: True
 - uncompetitive product inhibition model
 -- Fitting succeeded: True
 - uncompetitive product inhibition with enzyme inactivation model
 -- Fitting succeeded: True
 - non-competitive product inhibition model
 -- Fitting succeeded: True
 - non-competitive product inhibition with enzyme inactivation model
 -- Fitting succeeded: True
 - substrate inhibition model
 -- Fitting succeeded: True
 - substrate inhibition with enzyme inactivation model
 -- Fitting succeeded: True


,AIC,RMSD,kcat [1/min],Km [mmol / l],kcat / Km [1/min * 1/mmol / l],ki time-dep enzyme-inactiv. [1/min],Ki competitive [mmol / l],Ki uncompetitive [mmol / l]
substrate inhibition with enzyme inactivation,-2737,0.004310,3.0858 +/- 2.30%,1.1009 +/- 3.39%,2.803 +/- 4.10%,0.0325 +/- 4.37%,-,11.9412 +/- 7.27%
uncompetitive product inhibition with enzyme inactivation,-2517,0.006669,2.5702 +/- 6.56%,0.8822 +/- 12.57%,2.913 +/- 14.18%,0.0189 +/- 28.94%,-,0.5227 +/- 47.78%
non-competitive product inhibition with enzyme inactivation,-2515,0.006669,2.5704 +/- 6.62%,0.8823 +/- 12.77%,2.913 +/- 14.38%,0.0189 +/- 29.08%,999.9847 +/- 270.94%,0.5223 +/- 48.03%
irreversible Michaelis Menten with enzyme inactivation,-2510,0.006788,2.2355 +/- 1.68%,0.6498 +/- 2.34%,3.440 +/- 2.88%,0.0324 +/- 6.91%,-,-
competitive product inhibition with enzyme inactivation,-2508,0.006788,2.2355 +/- 1.70%,0.6497 +/- 2.53%,3.441 +/- 3.05%,0.0324 +/- 6.94%,983.1988 +/- 4125.10%,-
uncompetitive product inhibition,-2505,0.006853,3.2530 +/- 6.22%,1.3846 +/- 7.59%,2.349 +/- 9.82%,-,-,0.1638 +/- 13.72%
non-competitive product inhibition,-2503,0.006853,3.2527 +/- 6.24%,1.3843 +/- 7.62%,2.350 +/- 9.85%,-,509.0032 +/- 42.49%,0.1639 +/- 13.76%
substrate inhibition,-2434,0.007892,2.4262 +/- 3.81%,1.0912 +/- 6.23%,2.223 +/- 7.30%,-,-,12.0827 +/- 13.43%
competitive product inhibition,-2349,0.009347,1.8118 +/- 1.52%,0.5983 +/- 5.00%,3.028 +/- 5.22%,-,0.5276 +/- 47.82%,-
irreversible Michaelis Menten,-2346,0.009447,1.7658 +/- 1.00%,0.6473 +/- 3.26%,2.728 +/- 3.41%,-,-,-


In [344]:
estimator.visualize_model_overview()

y0s: [[0.2408292201794229, 0.009191176471, 0.0091707798205771, 0.0], [0.4856678535918146, 0.009191176471, 0.0143321464081854, 0.0], [0.0944761320831287, 0.009191176471, 0.0055238679168713, 0.0], [2.470074525249285, 0.009191176471, 0.0299254747507152, 0.0], [0.9760373674391369, 0.009191176471, 0.0239626325608631, 0.0], [4.971591509461829, 0.009191176471, 0.0284084905381707, 0.0], [0.9783534085541477, 0.009191176471, 0.0216465914458523, 0.0], [0.0940712833029243, 0.009191176471, 0.0059287166970757, 0.0], [4.973094199767816, 0.009191176471, 0.0269058002321838, 0.0], [0.4839042453856469, 0.009191176471, 0.0160957546143531, 0.0], [0.2401530562474708, 0.009191176471, 0.0098469437525292, 0.0], [2.4725349044743523, 0.009191176471, 0.0274650955256476, 0.0]]
y0s: [[0.2408292201794229, 0.009191176471, 0.0091707798205771, 0.0], [0.4856678535918146, 0.009191176471, 0.0143321464081854, 0.0], [0.0944761320831287, 0.009191176471, 0.0055238679168713, 0.0], [2.470074525249285, 0.009191176471, 0.02992547

In [ ]:
data = estimator.subset_substrate

inits = estimator.subset_initial_substrate
list(set(inits))

[0.1, 0.25, 2.5, 0.5, 1.0, 5.0]

In [ ]:
import numpy as np

In [ ]:
data = estimator.subset_product
data = data.reshape(6, 2, -1)

means = np.mean(data, axis=1)
stds = np.std(data, axis=1)

In [ ]:
means

array([[0.00572629, 0.00829216, 0.01079402, 0.01309609, 0.01526546,
        0.01723371, 0.01920413, 0.02104061, 0.02281083, 0.02444649,
        0.02601532, 0.02738073, 0.02881544, 0.03004614, 0.03120927,
        0.03244163, 0.03346927, 0.03449745, 0.03538901, 0.03621242,
        0.03717361],
       [0.00950886, 0.01506214, 0.01961199, 0.0235602 , 0.02785876,
        0.03155133, 0.03566334, 0.03950994, 0.04302033, 0.04633059,
        0.04950878, 0.05248496, 0.0553273 , 0.05796573, 0.06039944,
        0.06290629, 0.06527736, 0.06737188, 0.06974908, 0.07170911,
        0.07367114],
       [0.01521395, 0.02323593, 0.03156784, 0.03911367, 0.04627787,
        0.05291699, 0.05985874, 0.06633964, 0.07207411, 0.07747636,
        0.08303613, 0.0884018 , 0.09357154, 0.09832995, 0.10281643,
        0.10717176, 0.11132321, 0.115413  , 0.11944065, 0.12311705,
        0.12680148],
       [0.02280461, 0.03572588, 0.04812074, 0.05962832, 0.07135263,
        0.08301673, 0.09312739, 0.10315532, 0.1120938

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.colors

In [ ]:
def color_to_RGBA_string(color: list):
    return f"rgba{tuple(color)}"

In [ ]:
def _visibility_mask(visible_traces: list, fig_data: list) -> list:
    return [trace.name in visible_traces for trace in fig_data]

In [275]:
fig = go.Figure()
colors = matplotlib.colors.to_rgba_array(px.colors.qualitative.Plotly)

for mean, std, time, color_list in zip(means, stds, estimator.subset_time, colors):
    color = color_to_RGBA_string(color_list)
    fig.add_trace(
        go.Scatter(name="Data", x=time, y=mean, line=dict(color=color), mode="markers")
    )
    fig.add_trace(
        go.Scatter(
            name="Bound",
            x=time,
            y=mean + std,
            mode="lines",
            line=dict(width=0),
            showlegend=False,
        )
    )
    color_list[-1] = 0.2
    color = color_to_RGBA_string(color_list)
    fig.add_trace(
        go.Scatter(
            name="Bound",
            x=time,
            y=mean - std,
            line=dict(width=0),
            mode="lines",
            fillcolor=color,
            fill="tonexty",
            showlegend=True,
        )
    )

um = [{}, {}]
buttons = []
menuadjustment = 0.15

buttonX = -0.1
buttonY = 1 + menuadjustment

buttons = []
buttons.append(
    dict(
        method="update",
        label="show std",
        visible=True,
        args=[
            {
                "visible": _visibility_mask(
                    visible_traces=["Data", "Bound"], fig_data=fig.data
                )
            }
        ],
    )
)

buttons.append(
    dict(
        method="update",
        label="data only",
        visible=True,
        args=[{"visible": [trace.name != "Bound" for trace in fig.data]}],
    )
)

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.7,
            y=1.3,
            showactive=True,
            buttons=buttons,
        )
    ]
)


fig.show()

In [ ]:
print(typenp.array(y_lower)[::-1])
np.array(y_lower)

NameError: name 'typenp' is not defined

In [ ]:
import matplotlib.colors

matplotlib.colors.to_rgba_array(colors)

array([[0.38823529, 0.43137255, 0.98039216, 1.        ],
       [0.9372549 , 0.33333333, 0.23137255, 1.        ],
       [0.        , 0.8       , 0.58823529, 1.        ],
       [0.67058824, 0.38823529, 0.98039216, 1.        ],
       [1.        , 0.63137255, 0.35294118, 1.        ],
       [0.09803922, 0.82745098, 0.95294118, 1.        ],
       [1.        , 0.4       , 0.57254902, 1.        ],
       [0.71372549, 0.90980392, 0.50196078, 1.        ],
       [1.        , 0.59215686, 1.        , 1.        ],
       [0.99607843, 0.79607843, 0.32156863, 1.        ]])

In [ ]:
a = color

a[-1] = 11

a = tuple(a)

s = f"rgba{a}"
s

'rgba(0.38823529411764707, 0.43137254901960786, 0.9803921568627451, 11.0)'

In [ ]:
def color_to_RGBA_string(color: list):
    return f"rgba{tuple(color)}"

In [ ]:
color_to_RGBA_string(color)

'rgba(0.09803921568627451, 0.8274509803921568, 0.9529411764705882, 1.0)'

In [ ]:
"rgba(0.38823529411764707, 0.43137254901960786, 0.9803921568627451, 0.5)"

In [ ]:
_visibility_mask(trace_names=["Bound", "Data"], fig_data=fig.data)

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [ ]:
fig = go.Figure()
colors = px.colors.qualitative.Plotly

datas = estimator.subset_product.reshape(6, 2, -1)
times = estimator.subset_time.reshape(6, 2, -1)

for i, (data, time) in enumerate(zip(datas, times)):
    for j, (replicate_data, replicate_time) in enumerate(zip(data, time)):
        show_legend = True if j == 0 else False
        print(f"c: {colors[i]}")
        fig.add_trace(
            go.Scatter(
                x=replicate_time,
                y=replicate_data,
                name=f"raw",
                mode="markers",
                marker=dict(color=colors[i]),
                showlegend=show_legend,
                customdata=["raw"],
                hoverinfo="skip",
            )
        )

fig.show()

c: #636EFA
c: #636EFA
c: #EF553B
c: #EF553B
c: #00CC96
c: #00CC96
c: #AB63FA
c: #AB63FA
c: #FFA15A
c: #FFA15A
c: #19D3F3
c: #19D3F3


In [ ]:

        # Integrate each successfully fitted model
        successful_models = []
        for model in self.models.values():
            if model.result.fit_success:
                successful_models.append(model.name)
                datas = model.integrate(
                    model._fit_result.params,
                    self.subset_time,
                    self._get_unique_y0s(model.y0),
                )
                for j, model_data in enumerate(
                    datas[:, :, species_tuple]
                ):  # plots substrate
                    fig.add_trace(
                        go.Scatter(
                            x=self.subset_time[i],
                            y=model_data,
                            name=model.name,
                            marker=dict(color=colors[i]),
                            customdata=[f"{model.name}"],
                            showlegend=False,
                            hoverinfo="name",
                        )
                    )

        # Create selection options for dropdown menu
        buttons = []
        data_traces = [
            trace["customdata"][0] == "measurement_data" for trace in fig.data
        ]
        buttons.append(
            dict(
                method="restyle",
                label="Data",
                visible=True,
                args=[{"visible": data_traces}],
            )
        )

        # Create dropdown option for each fitte model
        for name in successful_models:
            visible_traces = [
                name == trace["customdata"][0]
                or "measurement_data" == trace["customdata"][0]
                for trace in fig.data
            ]
            buttons.append(
                dict(
                    method="restyle",
                    label=name,
                    visible=True,
                    args=[{"visible": visible_traces}],
                )
            )

        buttons.append(
            dict(
                method="restyle",
                label="All",
                visible=True,
                args=[{"visible": [True] * len(fig.data)}],
            )
        )

        # placement of dropdown menue
        fig.update_layout(
            updatemenus=[
                dict(
                    buttons=buttons,
                    x=0.0,
                    xanchor="left",
                    y=1.15,
                    yanchor="top",
                    direction="down",
                    showactive=True,
                )
            ]
        )


In [335]:
plots = ["ab", "bc", "cd"]
labels = ["bc", "cd"]


def check_lists(labels, plots):
    return [any(plot == label for label in labels) for plot in plots]


check_lists(labels, plots)

[False, True, True]